<a href="https://colab.research.google.com/github/jifarquharson/Farquharson_Amelung_2020_Kilauea-Nature/blob/master/Rainfall-gauge-analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [2]:
'''
Function converts mm input to inches (for plotting figures the correct size).
'''

def mm2inch(*tupl):
    if isinstance(tupl[0], tuple):
        return tuple(k*0.0393701 for k in tupl[0])
    else:
        return tuple(k*0.0393701 for k in tupl)

In [3]:
'''
Read in NCDC gauge data (accesses the GitHub project where we have uploaded the gauge data)
'''
all_gauges = pd.read_csv("https://raw.githubusercontent.com/jifarquharson/Farquharson_Amelung_2020_Kilauea-Nature/master/gauge_data/all_gauges_.csv")
all_gauges.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,DAPR,MDPR,PRCP,SNOW,SNWD
0,USC00517023,"KEAAU 92, HI US",19.63333,-155.03333,85.0,1930-01-01,NaN,NaN,0.0,NaN,NaN
1,USC00517023,"KEAAU 92, HI US",19.63333,-155.03333,85.0,1930-01-02,NaN,NaN,3.0,NaN,NaN
2,USC00517023,"KEAAU 92, HI US",19.63333,-155.03333,85.0,1930-01-03,NaN,NaN,5.8,NaN,NaN
3,USC00517023,"KEAAU 92, HI US",19.63333,-155.03333,85.0,1930-01-04,NaN,NaN,2.3,NaN,NaN
4,USC00517023,"KEAAU 92, HI US",19.63333,-155.03333,85.0,1930-01-05,NaN,NaN,0.0,NaN,NaN


In [4]:
'''
One of the gauges is in a separate file. Read it in and add to the dataframe.
'''
keaau_2 = pd.read_csv("https://raw.githubusercontent.com/jifarquharson/Farquharson_Amelung_2020_Kilauea-Nature/master/gauge_data/keaau_2.csv")
all_gauges = all_gauges.append(keaau_2)

In [5]:
datetimes = []
for j in all_gauges.DATE:
    datetimes.append(dt.datetime.strptime(j, "%Y-%m-%d"))
all_gauges['dates'] = datetimes

gauge_list = [name for name in list(set(all_gauges.NAME))]
stn_list = [stn for stn in list(set(all_gauges.STATION))]
print("List of stations covered: {}".format(stn_list))

List of stations covered: ['USC00516552', 'US1HIHI0060', 'USC00517457', 'USW00021504', 'USC00517023', 'US1HIHI0055', 'USC00511303', 'US1HIHI0008', 'US1HIHI0051', 'USC00513872', 'US1HIHI0003']


In [6]:
'''
Separate gauges into individual dataframes based on name or number
'''

kuristown = all_gauges[all_gauges.NAME == 'KURTISTOWN 6.6 SSE, HI US']
pahoa_1 = all_gauges[all_gauges.NAME == 'PAHOA 7.5 S, HI US']
pahoa_2 = all_gauges[all_gauges.NAME == 'PAHOA 2.7 SSE, HI US']
pahoa_3 = all_gauges[all_gauges.NAME == 'PAHOA 2.1 E, HI US']
pahoa_4 = all_gauges[all_gauges.NAME == 'PAHOA 65, HI US']
hilo = all_gauges[all_gauges.NAME == 'HILO INTERNATIONAL AIRPORT 87, HI US']
keaau_1 = all_gauges[all_gauges.STATION == 'USC00517023']
keaau_2 = all_gauges[all_gauges.STATION == 'USC00513872']
mt_view_1 = all_gauges[all_gauges.NAME == 'MOUNTAIN VIEW 4.5 NNE, HI US']
mt_view_2 = all_gauges[all_gauges.NAME == 'MOUNTAIN VIEW 91, HI US']
hvnp_1 = all_gauges[all_gauges.NAME == 'HAWAII VOL. NATIONAL PARK HQ. 54, HI US']
keaau = keaau_1.append(keaau_2, ignore_index=True) ## append k1 and k2

In [ ]:
'''
List of all dataframes for iteration
'''
df_list = [kuristown, pahoa_1, pahoa_2, pahoa_3, pahoa_4, hilo, keaau_1,keaau_2, mt_view_1, mt_view_2, hvnp_1, keaau]

In [ ]:
'''
Plot daily rainfall. Note that this may take ten minutes or so to execute.
'''

fig = plt.figure(figsize=mm2inch(180, 300), dpi = 100) 
for ix, df in enumerate(df_list):
    ax = fig.add_subplot(len(df_list), 1, ix+1)
    ax.bar(df.dates, df.PRCP, ec = "k")
    label = [df.STATION.values[0]+" ({})".format(df.NAME.values[0]), ""]
    ax.set_title(label, loc = 'left', fontsize = "small")
plt.tight_layout()

In [7]:
def plot_rainfall_timeseries(axis, df, interval = "30d", label = False, notes = False, unit_scaler = 1):
    ax = axis
    index = df.dates ## Use the dates as the index
    series = pd.Series(df.PRCP.fillna(0).values, index = index) ## Replace NaN ("not a number") values with 0
    interval = interval
    data = [x/unit_scaler for x in series.rolling(interval).sum()] ## Unit scaler can be used to convert between daily/weekly/monthly average and mm/inches/m etc.
    ax.plot(index, data,  alpha= 1, color = "w", lw = 1.5, zorder = np.inf) ## Plot timeseries data
    ax.plot(index, data,  alpha= 1, color = blue, lw = 0.8, zorder = np.inf)

    ## Prettify plot ##
    ax.spines["top"].set_visible(False) 
    ax.spines["right"].set_visible(False)
    ax.yaxis.set_tick_params(labelsize="x-small")
    ax.tick_params(axis='both', which='major', labelsize="x-small", direction="out")
    ax.set_xlim(xmin = "2010-08-01")

    ## Retrieve mean and standard deviations of data ##
    m, std_1, std_1_, std_2, std_2_ = descriptive_stats(df, l = 30)

    x_val = "2018-08-01"

    ax.vlines(x = x_val, ymin = std_2_/unit_scaler, ymax=std_2/unit_scaler,color = blue, lw =.8)

    for y in [std_1, std_1_, std_2, std_2_]:
        ax.hlines(xmin = "2018-07-15", xmax = "2018-08-15", y = y/unit_scaler,
                 color = blue, lw =.8)
    ax.hlines(xmin = "2018-07-01", xmax = "2018-09-01", y = m/unit_scaler,
                 color = blue, lw =.8)
   
    ax.axhspan(ymin = std_1_/unit_scaler, ymax =std_1/unit_scaler, color= red,alpha = .15, lw =0)
    ax.axhspan(ymin = std_2_/unit_scaler, ymax =std_2/unit_scaler, color= red,alpha = .15, lw =0)
    ax.axhline(m/unit_scaler,color= red,alpha = .15, lw =.8)
    ticks_y = ticker.FuncFormatter(lambda x, pos: '{:.0f}'.format(x))
    ax.yaxis.set_major_formatter(ticks_y)
    if label == True:
        label = df.STATION.values[0]+" ({})".format(df.NAME.values[0].split()[0])
        ax.annotate("{}".format(label),
                    xy = (0,1), xytext = (12,-8),
                    xycoords = 'axes fraction', textcoords='offset points',
                   fontsize = 8, va = "center", ha = "left")
    if notes == True:
#         for i, y in enumerate([m, std_1, std_1_, std_2, std_2_]):
#             ax.annotate(r"${}$".format(
#                 ["\overline{x}",r"+1\varsigma",r"-1\varsigma",r"+2\varsigma",r"-2\varsigma" ][i]
#             ),
#                        xy = ("2018-10-01", y/300),xytext = (10,0),textcoords="offset points",
#                         va = "center",ha = "right", fontsize = "xx-small")
     for i, y in enumerate([m, std_1, std_2,]):
            ax.annotate(r"${}$".format(
                ["\overline{x}",r"+1\varsigma",r"+2\varsigma",][i]
            ),
                       xy = ("2018-10-01", y/unit_scaler),xytext = (10,0),textcoords="offset points",
                        va = "center",ha = "center", fontsize = "xx-small")

In [ ]:
plot_rainfall_timeseries(axis=axs[0], df=hilo, interval = "30d", label = True, notes = True, unit_scaler=300)

plot_rainfall_timeseries(axis=axs[1], df=pahoa_3, interval = "30d", label = True, notes = True, unit_scaler=300)